In [ ]:
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('..')
os.environ.update(dict(CUDA_VISIBLE_DEVICES='3'))

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm

from torch.distributions import MultivariateNormal
from models.Res import ResNet, resnet50
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from analysis import *

def hook(module, args, output):
    module._output = output.data
    return output

In [ ]:
model = resnet50(pretrained=True).cuda()
model.eval()

In [ ]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])

transform = T.Compose(
    [T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        normalize,
        ])

dataset = ImageFolder('/ssd1/datasets/corruptions/ImageNet-C/gaussian_noise/5/', transform=transform)
loader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=16)


In [ ]:
@torch.jit.script
def softmax_entropy(x: torch.Tensor) -> torch.Tensor:
    """Entropy of softmax distribution from logits."""
    return -(x.softmax(1) * x.log_softmax(1)).sum(1)


with torch.no_grad():
    entropys = []
    for i, (x, _) in enumerate(tqdm(loader)):
        x = x.cuda()
        y = model(x)
        e = softmax_entropy(y)
        entropys.append(e)

    entropys = torch.concat(entropys)

In [ ]:
torch.save(entropys, 'entropys.pth')

In [ ]:
df = pd.DataFrame(entropys.cpu().numpy())

In [ ]:
df.to_csv('entropys.csv')

In [ ]:
df[df<0.5*np.log(1000)].count()

In [ ]:
df.hist()